In [ ]:
# Lets start with a simple fitting problem
# That is fit a straight line using scipy and then with machine learning.
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
m = 2
b = -3
y = [m*k + b for k in x]
x, y

In [ ]:
# Add a little noise to the data
import numpy as np
import numpy.random as random
import math
import time

time.time()
random.seed(int(time.time()))
mu, sigma = 0, 0.15
dy = random.default_rng().normal(mu, sigma, size=len(y))
y = y + dy
y, dy

In [ ]:
# Lets first look at our data
%matplotlib inline
import matplotlib.pyplot as plt
plt.scatter(x,y)
plt.show()

In [ ]:
# Now we'll fit this using the scipy way - effectively a least squares fit
from scipy import optimize
def fnc(xx, m, b):
    return m*xx + b
res = optimize.curve_fit(fnc, x, y, p0=(1,1), full_output=False)
res

In [ ]:
# this tells us our slope is around 2, and our intercept is around -3 as expected.
# Plotting these results, along with the orginal data, gives this

plt.scatter(x,y)
m0,b0 = res[0]   # Use the slope and the intercept that we calculated
xout = [0, 5, 11]

def f(ecks):
    return fnc(ecks, m0, b0)

yout = list(map(f, xout))
plt.plot(xout, yout, linestyle='dashed', color='orange')
plt.show()

In [ ]:
# Now lets do this the ML way. We'll use the 'mean_squared_error' for consistency
import tensorflow as tf

(x_train, y_train) = np.array(x), np.array(y)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=(1,))
])

# Use a mean_squared_error as the loss, and using the default optimizer, Steepest Gradient Descent.
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss='mean_squared_error',
              metrics='accuracy')

# We explicitly set metrics to None as there are no appropriate metrics embedded in Keras to use for this fit.
# Most of them are categorical, which is for 

model.fit(x_train, y_train, epochs=50)

In [ ]:
# Now lets plot these results
y_pred = model.predict(xout)
plt.scatter(x,y)
plt.plot(xout, y_pred, linestyle='dashed', color='orange')
plt.show()

# Not quite as good as we'd like to see, but not random. Lets see if we can do better!

In [ ]:
# We'll allow it to train with more cycles, say 500
model.fit(x_train, y_train, epochs=500)

y_pred = model.predict(xout)
plt.scatter(x,y)
plt.plot(xout, y_pred, linestyle='dashed', color='orange')
plt.show()

# That should be much better

In [ ]:
# This is a one node neural network.
# In general, each point should have a matrix M and then adds a vector b.
# y = Mx + b

# Since we have only one point we have a 1x1 matrix.
# Can we introspect those weights.
first_layer_weights = model.layers[0].get_weights()
first_layer_weights

# Looking at these results implies a slope of about 2 and a y-intercept of about -3.

In [ ]:
# Note also that the accuracy that is shown is not changing, it always has a value of 0.0000e+00, i.e. 0.
# This is because the 'accuracy' metric only applies to classification problems, and not to continuous variable results.

# For the latter we can define a new metric using the keras.backend
# rmse_accuracy as 100-rmse : root mean squared error (rmse) for regression (only for Keras tensors)
def rmse_accuracy(y_true, y_pred):
    from keras import backend
    return (np.ones(backend.int_shape(y_pred)[0]) *  100) - backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
                                                                          
# This will return 100% minus the RMSE errror, as our regression analogy to accuracy,
# which will make our results look more appealing and give us more confidence that things are working.
                                                                              
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss='mean_squared_error',
              metrics=rmse_accuracy)                                     
    
model.fit(x_train, y_train, epochs=500)  # Again, using 500 epochs
y_pred = model.predict(xout)

plt.scatter(x,y)
plt.plot(xout, y_pred, linestyle='dashed', color='orange')
plt.show()                                                                       

In [ ]:
# We can now see the accuracy increasing as the loss is decreasing.

# Try this with more data and a more complex model, and a few more points
from tensorflow.keras import layers

xx = [x for x in range(-15,16)]
m = 2
b = -3
yy = [m*k + b for k in xx]

XX = np.array(xx)

print(f"len(xx) = {len(xx)}")

(x_train, y_train) = XX, np.array(yy)

model = tf.keras.Sequential()

# Add the first layer (input layer)
model.add(layers.Dense(units=1, activation='linear', input_shape=(1,)))

##Add another layer (hidden layer)
model.add(layers.Dense(units=3, activation='linear'))

# Add yet another layer (output layer)
model.add(layers.Dense(units=1, activation=None))  # Gives a simple number.

tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)

# Use a mean_squared_error as the loss, and using the default optimizer, Steepest Gradient Descent.
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.0004),
              loss='mean_squared_error',
              metrics=rmse_accuracy)

# We explicitly set metrics to None as there are no appropriate metrics embedded in Keras to use for this fit.
# Most of them are categorical, which is for 

print(f"model.summary = {model.summary()}")

model.fit(x_train, y_train, epochs=800)

In [ ]:
# Taking a look at the weights...
for i in range(3):
    layer_i_weights = model.layers[i].get_weights()
    print(f"layer_{i}_weights = {layer_i_weights}")
    
print('\n')
    
# ...We see it is no longer obvious how our slope and intercept got embedded in these 12 parameters (see output).
# In addition, the activation funtions were either None or linear, and this is because if we chose the more tradition ones
# like 'relu' or 'tanh' the results would have difficulty converging. 
# It was also necessary in this case to decrease the learning rate to make sure it converged. You can see this by
# resetting the learning rate to 0.1, and see what happens.

# We'll need to start thinking of normalizing our features and regulizing our training for 
# stable results in more general situations
# Finally we need to think about how many trainable parameters we have relative to our total data points.
# In this case we have 31 data points, and Trainable params of 12. 
# It's good to have # data points >> # of trainable params

# For completion, lets plot these results
plt.scatter(x_train, y_train)

x_out = [-14, -7, 1, 7, 16]
y_pred = model.predict(x_out)

plt.plot(x_out, y_pred, linestyle='dashed', color='orange')
plt.show()

In [ ]:
# Now lets solve the same problem using another framework for ML

# For example, lets try scikit learn - specifically scikit.models.LinearRegression
import math
import numpy as np  # note: was already imported
from sklearn.model_selection import train_test_split  # Often used with other frameworks 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Prepare your data, using capital X as out input data (as opposed to our original 'x')
# Using y = mX + b, with m = 2.0, and b = -3.0
# X values are referred to as 'Features'
X = np.array([[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]])

# Sample target values
y = [m*x + b for x in X.flatten()]
y = y + dy  # dy, from before, a bit of noise.

# This next step is not necessary, but almost always done.
# If we know we have a line, we could get the best estimate of the fitted parameters m & b, by using all the points.
# However, In genearal, we want to compare model and see what works best,
#    and we seldom know for sure that we have a perfect linear relationship,
#       so we need to set aside some portion of our data for testing.

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=2, random_state=3428)

# Create the regression model.
model = LinearRegression()

# NOTE: no compile step here. 
# Tensorflow often involves complicated architectures, and the compilation step allows for some graph optimization
# to allow for a faster fit step, which are not needed for the simpler models in scikit-learn.

# Fit the model to the training data
model.fit(X_train, y_train)  # not showing the fitting process.

# Make predictions
y_predX = model.predict(X)

# Evaluate the fit and print results
mse = mean_squared_error(X, y_predX)
RMSE = math.sqrt(mse/len(y_predX))  
r2 = r2_score(y, y_predX)

print("Scikit-learn Regression results:")
print(f"  Root Mean Squared Error: {round(RMSE, 6)}, R-squared: {round(r2, 6)}")

# Output fitting variables: m0, b0
m0 = model.coef_
b0 = model.intercept_
print(f"  slope:{np.round(m0, 3)}, intercept: {np.round(b0, 3)}, should be close to expectations")

In [ ]:
# Note: During fitting, this model does not have any output to show us it's progress toward a satisfactory model.
# There is discussed on the web & chatGPT of the existance of a verbose flag, but that did not work above. 

# plot raw data and results
plt.scatter(np.ndarray.flatten(X),y)

# Make predictions using our xout. 
# We need to reshape our xout to be two dimensional as scikit-learn typcally uses 2d input data , 
# as opposed to tensorflow which is anywhere from 1 to many dimensional 
xout_skl = np.array(xout).reshape(3,1)

y_pred = model.predict(xout_skl)
plt.plot(xout, y_pred, linestyle='dashed', color='orange')
plt.show()


In [ ]:
# Moving on to lesson_02